In [7]:
import os

import streamlit as st
from dotenv import load_dotenv
from openai import OpenAI

from langchain_core.messages import HumanMessage, AIMessage

In [8]:


# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    st.error("API Key not found! Please check your environment variables.")

client = OpenAI(api_key=api_key)

# Avatar URLs
user_avatar_url = "https://api.dicebear.com/8.x/pixel-art/svg?seed=Simon&beardProbability=0&eyesColor=876658&glassesProbability=0&hairColor=28150a&skinColor=cb9e6e"

specialist_id_caption = {
    "Diana's assistant": {
        "assistant_id": "asst_d1mG01RGb8UEA1Anue9QgkQF",
        "caption": "role is multifaceted, encompassing elements of an assistant, AI journal, therapist, friend, and counselor.",
        "avatar": "https://api.dicebear.com/8.x/bottts/svg?seed=Lucky"
    },
    "Mindfulness Teacher": {
        "assistant_id": "asst_bnFm27eqedaYK9Ulekh8Yjd9",
        "caption": "Goes Deep",
        "avatar": "https://cdn.pixabay.com/photo/2013/07/12/19/30/enlightenment-154910_1280.png"
    }
}

In [9]:
def initialize_session_state():
    primary_specialist = list(specialist_id_caption.keys())[0]
    primary_specialist_id = specialist_id_caption[primary_specialist]["assistant_id"]
    state_keys_defaults = {
        "chat_history": [],
        "user_question": "",
        "json_data": {},
        "critical_actions": {},
        "sidebar_state": 1,
        "assistant_response": "",
        "specialist_input": "",
        "specialist": primary_specialist,
        "assistant_id": primary_specialist_id,
        "should_rerun": False
        
    }
    for key, default in state_keys_defaults.items():
        if key not in st.session_state:
            st.session_state[key] = default

In [10]:
def display_header():
    st.set_page_config(page_title="DA", page_icon="🫡")
    specialist = st.session_state.specialist
    specialist_avatar = specialist_id_caption[st.session_state.specialist]["avatar"]
    st.markdown(
            f"""
            <div style="text-align: center;">
                <h1>
                    <img src="{specialist_avatar}" alt="Avatar" style="width:60px;height:60px;border-radius:50%;">
                    Diana's Assistant
                </h1>
            </div>
            """, 
            unsafe_allow_html=True
        )
        

In [11]:
def display_chat_history():
    chat_container = st.container()
    with chat_container:
        for message in st.session_state.chat_history:
            avatar_url = message.get("avatar", user_avatar_url)
            with st.chat_message(message["role"], avatar=avatar_url):
                st.markdown(message["content"], unsafe_allow_html=True)


In [16]:
def handle_user_input():
    user_question = input("How may I help you?")
    if user_question is not None and user_question != "":
        st.session_state.chat_history.append(HumanMessage(user_question, avatar=user_avatar_url))

        with st.chat_message("Human"):
            print(user_question)
        
        with st.chat_message("AI"):
            ai_response = "I don't know"
            print(ai_response)
   

In [17]:

def main():
    if "thread_id" not in st.session_state:
        thread = client.beta.threads.create()
        st.session_state.thread_id = thread.id
        
    initialize_session_state()
    display_header()
    handle_user_input()
    #display_chat_history()


In [18]:
handle_user_input()